# Spans
> Span class and how to interface it with pandas

In [ ]:
#| default_exp span

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from abc import ABC, abstractmethod
import pytest
from pathlib import Path
import pandas as pd
from pathlib import Path
from typing import no_type_check, Set, Sequence, Any,Optional,List,Callable,Dict,Union
from pydantic import BaseModel

pd.options.future.infer_string = True


I followed [this guide](https://itnext.io/guide-to-pandas-extension-types-and-how-to-create-your-own-3b213d689c86) on how to make extension types for pandas

In [ ]:
#| export
import hashlib

def small_hash(txt,length=6):
    """A function that returns a small hash of a string

    Args:
        txt (_type_): string to hash
        length (int, optional): length of hash. Defaults to 6.

    Returns:
        _type_: _description_
    """
    return hashlib.sha1(txt.encode()).hexdigest()[:length]

In [ ]:
#| export
SPAN_REPR_FORMAT = '''[@{doc},{start},{end}) "{text}"'''
SPAN_TEXT_HEAD_NUM = 10

def set_span_repr_format(format=None,head:int=None):
    """
    Sets the representation format for spans and the number of characters to display in the span text.

    Parameters:
        format (str, optional): The representation format for spans. Defaults to None.
        head (int, optional): The number of characters to display in the span text. Defaults to None.
    """
    global SPAN_REPR_FORMAT, SPAN_TEXT_HEAD_NUM
    if format is not None:
        SPAN_REPR_FORMAT = format
    if head is not None:
        SPAN_TEXT_HEAD_NUM = head

def get_span_repr_format() -> str:
    """
    Returns the span representation format.

    Returns:
        (the span representation format, the number of characters to display in the span text)
    """
    return SPAN_REPR_FORMAT, SPAN_TEXT_HEAD_NUM


In [ ]:
#| export
from enum import Enum
from typing import Any
from pydantic import ConfigDict
from collections import UserString
# TODO from here turn all code into having spanner act as a string with slicing, change the display option to use repr
# and add a note on it in the tutorial and the tests here

# we will have an ie function that casts a span to its string for viewing while developing - TODO

# whether we get a document as a string or as a file, we assume that it remains immutable throughout the process - TODO explain
# a user can access the original document through the span interface (currently we dont do disk caching etc so it will just be a string and not a document class) - TODO explain

class Span(UserString):
    def __init__(self,doc,start=None,end=None,name=None):
        if isinstance(doc,Span):
            father = doc
            sub_span = doc.slice(start,end)
            self.doc = sub_span.doc
            self.start = sub_span.start
            self.end = sub_span.end
            self.name = sub_span.name
        
        else:
            if isinstance(doc,Path):
                self.doc = doc.read_text()
                self.name = doc.name
            else:
                if name is None:
                    name = small_hash(doc)
                self.name = name
                self.doc = doc
                
            if start is None:
                start = 0
            if end is None:
                end = len(self.doc)
            self.start = start
            self.end = end


        super().__init__(self.as_str())


    def __getitem__(self, key):
        if isinstance(key, slice):
            return self.slice(key.start,key.stop)
        return self.doc[self.start+key]
    
    def slice(self, start=None,end=None):
        if start is None:
            start = 0
        if end is None:
            end = len(self)
        if start < 0 or end < 0:
            raise ValueError(f'Negative indices not supported, got start: {start}, end: {end}')
        if start > end:
            raise ValueError(f'Start index greater than end index, got start: {start}, end: {end}')
        if end > len(self):
            raise ValueError(f'End index greater than length of span, got end: {end}, length: {len(self)}')
        return Span(self.doc,self.start+start,self.start+end,name=self.name)

    # @property
    # def data(self):
    #     return str(self)
    
    def __repr__(self):
        f_string,head_num = get_span_repr_format()
        text = self.doc[self.start:self.end]
        if len(text) > head_num:
            text = text[:head_num] + '...'
        return SPAN_REPR_FORMAT.format(doc=self.name,start=self.start,end=self.end,text=text)

    def __len__(self):
        return self.end-self.start

    def __str__(self):
        return self.as_str()

    def as_str(self):
        return self.doc[self.start:self.end]
    
    # # used for sorting `Span`s in dataframes
    def __hash__(self) -> int:
        return hash((self.doc,self.start, self.end))

    def __lt__(self, other) -> bool:
        return (self.doc, self.start, self.end) < (other.doc, other.start, other.end)

    def __eq__(self, value: object) -> bool:
        if isinstance(value, Span):
            return self.start == value.start and self.end == value.end and self.doc == value.doc
        elif isinstance(value, str):
            return self.as_str() == value
        else:
            return False

    def as_tuple(self):
        return (self.doc,self.start, self.end)

    @classmethod
    def from_val(cls,val):
        if isinstance(val,Span):
            return val
        if isinstance(val, (list, tuple)) and len(val) == 3:
            return Span(doc=val[0],start=val[1], end=val[2])
        raise ValueError('Invalid value to create Vector from: {}'.format(val))
    


def ie(s:Span)->(int,int):
    return s.start,s.end

In [ ]:
assert Span("aa",0,2) == "aa"

In [ ]:
doc = 'world'
df = pd.DataFrame([
    [Span('hello',0,5),1],
    [Span(doc,0,5),2],
    [Span(doc,0,5),3],
], columns=['span','num'])
df

,span,num
0,"(h, e, l, l, o)",1
1,"(w, o, r, l, d)",2
2,"(w, o, r, l, d)",3


In [ ]:
doc = 'world'
df = pd.DataFrame([
    ['hello',1],
    ['world',2],
    ['world',3],
], columns=['span','num'])
df

,span,num
0,hello,1
1,world,2
2,world,3


In [ ]:
#TODO from here, ok so We need union types and to make the span class print prettily
df.groupby('span').sum()

,num
span,
hello,1
world,5


In [ ]:
string = "hello stranger"
short_string = "hi"

In [ ]:
s = Span(string,0,len(string),name ='doc')
display(s)

[@doc,0,14) "hello stra..."

In [ ]:
pd.DataFrame({'span':[s]})

,span
0,"(h, e, l, l, o, , s, t, r, a, n, g, e, r)"


In [ ]:
df = pd.DataFrame({'span':[s]}).map(repr)
df

,span
0,"[@doc,0,14) ""hello stra..."""


In [ ]:
s2 = Span(short_string)
display(s2)

[@c22b5f,0,2) "hi"

In [ ]:
assert s == 'hello stranger'
assert s[0:5] == 'hello'
assert not s == s[0:5]
assert f"{s[0:5].as_str()} darkness" == 'hello darkness'
assert s[0:5][1:4] == 'ell'

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     